In [ ]:
import pandas as pd
training_set = pd.read_csv('./DATA/training+synthetic+OHE/training_set.csv')
cv_set = pd.read_csv('./DATA/CV set/cv_set.csv')
holdout_set = pd.read_csv('./DATA/holdout_set/test_set.csv')